# Sample 3: Cloud Deploy

Reference: cloud.google.com/deploy/docs/overview

## Deploy an app to GKE using Cloud Deploy

In [2]:
# gcloud init

In [5]:
!echo $(gcloud config get-value project)

training1027


In [4]:
# Make sure the default ompute Engine service account has sufficient permissions
# Add the clouddeploy.jobRunner role
!gcloud projects add-iam-policy-binding $(gcloud config get-value project) \
    --member=serviceAccount:$(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --role="roles/clouddeploy.jobRunner"

Updated IAM policy for project [training1027].
bindings:
- members:
  - serviceAccount:service-487762169828@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:487762169828@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/cloudbuild.builds.editor
- members:
  - serviceAccount:service-487762169828@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/clouddeploy.admin
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com
  role: roles/clouddeploy.jobRunner
- members:
  - serviceAccount:service-487762169828@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-487762169828@container-engine-robot.iam.gs

In [7]:
# Add the Kubernetes developer permissions
!gcloud projects add-iam-policy-binding $(gcloud config get-value project) \
    --member=serviceAccount:$(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --role="roles/container.developer"

Updated IAM policy for project [training1027].
bindings:
- members:
  - serviceAccount:service-487762169828@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:487762169828@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/cloudbuild.builds.editor
- members:
  - serviceAccount:service-487762169828@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/clouddeploy.admin
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com
  role: roles/clouddeploy.jobRunner
- members:
  - serviceAccount:service-487762169828@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com

In [8]:
# Add the iam.serviceAccountUser role, which includes the actAs permission to deploy to the runtime
!gcloud iam service-accounts add-iam-policy-binding $(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --member=serviceAccount:$(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --role="roles/iam.serviceAccountUser" \
    --project=$(gcloud config get-value project)

Updated IAM policy for serviceAccount [487762169828-compute@developer.gserviceaccount.com].
bindings:
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com
  - serviceAccount:487762169828@cloudbuild.gserviceaccount.com
  role: roles/iam.serviceAccountUser
etag: BwYIreLaEwQ=
version: 1


### Create Google Kubernetes Engine clusters

In [ ]:
!gcloud container clusters create-auto quickstart-cluster-qsdev \
    --project=$(gcloud config get-value project) \
    --region=us-west1 && gcloud container clusters create-auto quickstart-cluster-qsprod \
    --project=$(gcloud config get-value project) \
    --region=us-west1

Note: The Pod address range limits the maximum size of the cluster. Please refer to https://cloud.google.com/kubernetes-engine/docs/how-to/flexible-pod-cidr to learn how to optimize IP address allocation.
Creating cluster quickstart-cluster-qsdev in us-west1... Cluster is being confi
gured...⠼                                                                      
Creating cluster quickstart-cluster-qsdev in us-west1... Cluster is being deplo
yed...⠼                                                                        
Creating cluster quickstart-cluster-qsdev in us-west1... Cluster is being healt
h-checked...⠹                                                                  

### Prepare Skaffold configuration and Kubernetes manifest

In [ ]:
!mkdir deploy-gke-quickstart

In [ ]:
pwd

In [ ]:
cd deploy-gke-quickstart

In [ ]:
pwd

In [ ]:
%%writefile skaffold.yaml
apiVersion: skaffold/v2beta16
kind: Config
deploy:
  kubectl:
    manifests:
      - k8s-*

In [ ]:
%%writefile k8s-pod.yaml
apiVersion: v1
kind: Pod
metadata:
  name: getting-started
spec:
  containers:
  - name: nginx
    image: my-app-image

### Create your delivery pipeline and targets

In [ ]:
%%writefile clouddeploy.yaml
apiVersion: deploy.cloud.google.com/v1
kind: DeliveryPipeline
metadata:
  name: my-gke-demo-app-1
description: main application pipeline
serialPipeline:
  stages:
  - targetId: qsdev
    profiles: []
  - targetId: qsprod
    profiles: []
---

apiVersion: deploy.cloud.google.com/v1
kind: Target
metadata:
  name: qsdev
description: development cluster
gke:
  cluster: projects/PROJECT_ID/locations/us-central1/clusters/quickstart-cluster-qsdev
---

apiVersion: deploy.cloud.google.com/v1
kind: Target
metadata:
  name: qsprod
description: production cluster
gke:
  cluster: projects/PROJECT_ID/locations/us-central1/clusters/quickstart-cluster-qsprod